In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr scikit-learn

In [2]:
from fastkaggle import *

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#12) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv.7z'),Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv')...]

In [7]:
def is_extracted(file):
    return os.path.isfile(os.path.splitext(file)[0])

for file in path.ls(file_exts=".7z"):
    if not is_extracted(file):
        !py7zr x {str(file)} {str(path)}

In [8]:
path.ls(file_exts=".csv")

(#6) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/members.csv')]

In [9]:
songs = pd.read_csv(path/'train.csv', usecols=('msno', 'song_id', 'target'))

songs.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [10]:
song_data_1 = pd.read_csv(path/'song_extra_info.csv')

song_data_1.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [11]:
song_data_0 = pd.read_csv(path/'songs.csv')

song_data_0.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [12]:
members = pd.read_csv(path/'members.csv')

members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [13]:
with_song_0 = pd.merge(songs, song_data_0, how='left', on='song_id')
with_song_1 = pd.merge(with_song_0, song_data_1, how='left', on='song_id')
all_data = pd.merge(with_song_1, members, how='left', on='msno')
all_data

,msno,song_id,target,song_length,genre_ids,artist_name,composer,lyricist,language,name,isrc,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,Good Grief,GBUM71602854,1,0,NaN,7,20120102,20171005
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1,284584.0,1259,Various Artists,NaN,NaN,52.0,Lords of Cardboard,US3C69910183,13,24,female,9,20110525,20170911
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,Hip Hop Is Dead(Album Version (Edited)),USUM70618761,13,24,female,9,20110525,20170911
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,Disco Africa,GBUQH1000063,13,24,female,9,20110525,20170911
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,Sleep Without You,QM3E21606003,1,0,NaN,7,20120102,20171005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,1,267958.0,1259,Dr. Dre,Andre R. Young| Melvin Charles Bradford| Shawn Carter| Scott Storch,NaN,52.0,Still D.R.E.,USIR19905031,4,26,female,9,20041218,20170930
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,0,227404.0,465,R. City feat. Adam Levine,Theron Thomas| Timothy Thomas| Lukasz Gottwald| Henry Walter| Toni Tennille,Theron Thomas| Timothy Thomas| Lukasz Gottwald| Henry Walter| Toni Tennille,52.0,Locked Away,USRC11501369,1,0,NaN,7,20130627,20171006
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,1,258298.0,465,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,3.0,FLY OUT,TWA451512031,1,0,NaN,7,20130627,20171006
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,1,524146.0,2122,Brad Mehldau Trio,NaN,NaN,52.0,Wonderwall,USNO10750721,5,0,NaN,9,20150213,20170918


In [14]:
all_data = add_datepart(all_data, 'registration_init_time')
all_data = add_datepart(all_data, 'expiration_date')

all_data

/nix/store/z5kpw9gkd5hza93c8s34fg8hx4hbiy3s-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)
/nix/store/z5kpw9gkd5hza93c8s34fg8hx4hbiy3s-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


,msno,song_id,target,song_length,genre_ids,artist_name,composer,lyricist,language,name,...,expiration_Day,expiration_Dayofweek,expiration_Dayofyear,expiration_Is_month_end,expiration_Is_month_start,expiration_Is_quarter_end,expiration_Is_quarter_start,expiration_Is_year_end,expiration_Is_year_start,expiration_Elapsed
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,Good Grief,...,1,3,1,False,True,False,True,False,True,0.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1,284584.0,1259,Various Artists,NaN,NaN,52.0,Lords of Cardboard,...,1,3,1,False,True,False,True,False,True,0.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,Hip Hop Is Dead(Album Version (Edited)),...,1,3,1,False,True,False,True,False,True,0.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,Disco Africa,...,1,3,1,False,True,False,True,False,True,0.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,Sleep Without You,...,1,3,1,False,True,False,True,False,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,1,267958.0,1259,Dr. Dre,Andre R. Young| Melvin Charles Bradford| Shawn Carter| Scott Storch,NaN,52.0,Still D.R.E.,...,1,3,1,False,True,False,True,False,True,0.0
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,0,227404.0,465,R. City feat. Adam Levine,Theron Thomas| Timothy Thomas| Lukasz Gottwald| Henry Walter| Toni Tennille,Theron Thomas| Timothy Thomas| Lukasz Gottwald| Henry Walter| Toni Tennille,52.0,Locked Away,...,1,3,1,False,True,False,True,False,True,0.0
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,1,258298.0,465,兄弟本色G.U.T.S. (姚中仁、張震嶽、頑童MJ116),姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,姚中仁 MC HotDog 陳昱榕 Ｅ-SO 周文傑 KENZY 林睦淵,3.0,FLY OUT,...,1,3,1,False,True,False,True,False,True,0.0
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,1,524146.0,2122,Brad Mehldau Trio,NaN,NaN,52.0,Wonderwall,...,1,3,1,False,True,False,True,False,True,0.0


In [15]:
cat_names = ['msno', 'song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language', 'name', 'isrc', 'city', 'gender', 'registered_via', 'registration_init_timeYear', 'registration_init_timeMonth', 'registration_init_timeWeek', 'registration_init_timeDay', 'registration_init_timeDayofweek', 'registration_init_timeDayofyear', 'registration_init_timeIs_month_end', 'registration_init_timeIs_month_start', 'registration_init_timeIs_quarter_end', 'registration_init_timeIs_quarter_start', 'registration_init_timeIs_year_end', 'registration_init_timeIs_year_start', 'expiration_Year', 'expiration_Month', 'expiration_Week', 'expiration_Day', 'expiration_Dayofweek', 'expiration_Dayofyear', 'expiration_Is_month_end', 'expiration_Is_month_start', 'expiration_Is_quarter_end', 'expiration_Is_quarter_start', 'expiration_Is_year_end', 'expiration_Is_year_start']
cont_names = ['bd', 'song_length', 'registration_init_timeElapsed', 'expiration_Elapsed']
dep = 'target'

In [16]:
to = TabularPandas(all_data, procs=[Categorify, FillMissing,Normalize],
                   cat_names=cat_names,
                   cont_names=cont_names,
                   y_names=dep,
                   splits=RandomSplitter(valid_pct=0.2)(range_of(all_data)))

/nix/store/z5kpw9gkd5hza93c8s34fg8hx4hbiy3s-python3-3.11.9-env/lib/python3.11/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [17]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [19]:
to.train.ys.values.ravel?

Docstring:
a.ravel([order])

Return a flattened array.

Refer to `numpy.ravel` for full documentation.

See Also
--------
numpy.ravel : equivalent function

ndarray.flat : a flat iterator on the array.
Type:      builtin_function_or_method

In [22]:
to.train.ys.values

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int8)

In [17]:
to.xs.iloc[:2]

,msno,song_id,genre_ids,artist_name,composer,lyricist,language,name,isrc,city,...,expiration_Is_month_start,expiration_Is_quarter_end,expiration_Is_quarter_start,expiration_Is_year_end,expiration_Is_year_start,song_length_na,bd,song_length,registration_init_timeElapsed,expiration_Elapsed
6713476,28956,185320,121,19777,23892,0,9,75015,0,1,...,1,1,1,1,1,1,-0.814411,0.011306,0.0,0.0
4642796,21515,279784,345,13963,28100,9387,9,116260,14815,11,...,1,1,1,1,1,1,0.300240,-0.501172,0.0,0.0


In [18]:
dls = to.dataloaders(bs=512)
dls.show_batch()

,msno,song_id,genre_ids,artist_name,composer,lyricist,language,name,isrc,city,gender,registered_via,registration_init_timeYear,registration_init_timeMonth,registration_init_timeWeek,registration_init_timeDay,registration_init_timeDayofweek,registration_init_timeDayofyear,registration_init_timeIs_month_end,registration_init_timeIs_month_start,registration_init_timeIs_quarter_end,registration_init_timeIs_quarter_start,registration_init_timeIs_year_end,registration_init_timeIs_year_start,expiration_Year,expiration_Month,expiration_Week,expiration_Day,expiration_Dayofweek,expiration_Dayofyear,expiration_Is_month_end,expiration_Is_month_start,expiration_Is_quarter_end,expiration_Is_quarter_start,expiration_Is_year_end,expiration_Is_year_start,song_length_na,bd,song_length,registration_init_timeElapsed,expiration_Elapsed,target
0,Mn98O4X/zceMt9nswcoq08F16ciCqhuN7lQ951MAL2o=,rFLawM8j/vW78o/S3P2GTsmy6HwkTI15P7NF/i9CkX4=,465|458,林芯儀 (Shennio Lin),吳姝霆,張鵬鵬,3.0,以後以後 (A better tomorrow),TWX351600002,15,female,4,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,2.400000e+01,237504.999884,0.0,0.0,1.0
1,Zp3UvIu8dB7geU2GO4h7vJkU0sbayHw2yBo6Udp9K5I=,zzqFu2/dTaFzWZ7YdB7SN0aILlpbVoonyeULaLBW4mE=,465,Mariah Carey,#na#,#na#,52.0,Fantasy,USSM19501023,13,male,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,4.400000e+01,289320.000711,0.0,0.0,0.0
2,/zeGVN0jXVoNo1Zp3P/T+IZISOcMncYqxq+/p9Ba/7I=,KFr700pRPsYoDLQBvOfvBff7Qwz3NAtlElKGQnd5c6M=,465,蜜雪薇琪 (Michelle & Vickie),#na#,#na#,3.0,獨立,TWA130400401,8,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,2.500000e+01,200619.999957,0.0,0.0,0.0
3,XO4zzFgCoqtKCvSRiYHBaeJ0n1pYAK3YqeHGKwcMfZc=,NAP8r72c1JsKDAxqaaYcYmwZGPggGzzN+n5RbeKyaeE=,465,iKON,#na#,#na#,31.0,ANTHEM,#na#,1,#na#,9,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,-4.525706e-07,205270.000346,0.0,0.0,0.0
4,s0i69ExiixcmnNfEEtScP4ECGZZBJD3+BDGpsMKZr48=,Eh39FUG2Eod+wousJ2K6L9gcegFxfPGt0PgkM2NKeyc=,465,President of Pint,Public Domain,Public Domain,52.0,Ppap Pen-Pineapple-Apple-Pen,DEZ921600547,1,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,-4.525706e-07,109319.996171,0.0,0.0,0.0
5,OqLPHCE0NFDQrEw67zOqJcz9bU+mv4976hQMZw5lZn8=,0b8M2FLEH176y/ng1rK9ftUYDYnA+1NdoCOUeuyBMtU=,1609,The Chainsmokers,Andrew Taggart| Kris Eriksson| Fredrik Häggstam,Andrew Taggart| Kris Eriksson| Fredrik Häggstam,52.0,Paris,USQX91603031,13,female,9,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,2.400000e+01,221543.999069,0.0,0.0,1.0
6,5fPe5vTWrj5VZWmY4Yl0PB8oAc9WsvppqutinPUQjTU=,0vs+fijSql0WbMAWWEuSW7RwfKvY71mC+gK0rD2LhqA=,1609|359,DJ Snake & AlunaGeorge,George Reid| Aluna Francis,#na#,52.0,You Know You Like It,USUM71414293,1,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,-4.525706e-07,247245.999963,0.0,0.0,1.0
7,hxcZ8pjC3nXNtiU10IFHVtE/MJ6ftnTy0akt9IpuLEw=,vYgdg3j2g8cztFopB8f2FL/tm88UCKQn5FpNCXO3o/k=,1609|458,原子邦妮 (Astro Bunny),查查\Nu,查查,3.0,孤單會消失離開不見 (Loneliness will be over| away and gone),TWAE31601073,1,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,-4.525706e-07,244991.999997,0.0,0.0,0.0
8,T1mZ0wwVLTXk8bnqpKRMO5R3EPbQiKdGnbFATjgFDLk=,6KxOKdCMQeNgRSJr6BCzHZxJ2urR02KJN73pAFRfTdI=,139|125|109,Jayesslee,#na#,#na#,52.0,Officially Missing You,TCABF1297271,1,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,True,False,True,False,-4.525706e-07,222911.000200,0.0,0.0,1.0
9,16fsOt7imefLSFsV7mWoP1AL+bpjP1NuWzQ5QP9tj8k=,WG6Jc3TmXPoyieHBx+xRtJbXW7Ua1MzW9L5m022vOQs=,1609,Deadmau5,#na#,#na#,52.0,Let Go,GBTDG1301198,1,#na#,7,1970,1,1,1,3,1,False,True,False,True,False,True,1970,1,1,1,3,1,False,True,False,Tr

In [19]:
from sklearn.metrics import roc_auc_score

def roc_metric(pred, target):
    return roc_auc_score(to_np(target.squeeze()), to_np(pred))

learn = tabular_learner(dls, metrics=roc_metric)

In [20]:
learn.fit_one_cycle(5, 1e-1, wd=0.1)

epoch,train_loss,valid_loss,roc_metric,time
0,0.235418,0.235393,0.645544,52:26
1,0.233764,0.234850,0.650798,52:28
2,0.230832,0.235350,0.663262,52:27
3,0.218045,0.220924,0.700014,52:29
4,0.192054,0.200904,0.755703,52:28


In [21]:
torch.save(learn.model.state_dict(), '06.pth')
# learn.model.load_state_dict(torch.load('01.pth'))

In [22]:
test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))

test_with_song_0 = pd.merge(test_df, song_data_0, how='left', on='song_id')
test_with_song_1 = pd.merge(test_with_song_0, song_data_1, how='left', on='song_id')
test_all_data = pd.merge(test_with_song_1, members, how='left', on='msno')

test_all_data = add_datepart(test_all_data, 'registration_init_time')
test_all_data = add_datepart(test_all_data, 'expiration_date')

test_all_data.head()

,id,msno,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,name,...,expiration_Day,expiration_Dayofweek,expiration_Dayofyear,expiration_Is_month_end,expiration_Is_month_start,expiration_Is_quarter_end,expiration_Is_quarter_start,expiration_Is_year_end,expiration_Is_year_start,expiration_Elapsed
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,NaN,3.0,愛其實很殘忍,...,1,3,1,False,True,False,True,False,True,0.0
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,320470.0,465,林俊傑 (JJ Lin),林俊傑,孫燕姿/易家揚,3.0,她說,...,1,3,1,False,True,False,True,False,True,0.0
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,315899.0,2022,Yu Takahashi (高橋優),Yu Takahashi,Yu Takahashi,17.0,subarashiki nichijo,...,1,3,1,False,True,False,True,False,True,0.0
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,285210.0,465,U2,The Edge| Adam Clayton| Larry Mullen| Jr.,NaN,52.0,Hold Me| Thrill Me| Kiss Me| Kill Me,...,1,3,1,False,True,False,True,False,True,0.0
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,197590.0,873,Yoga Mr Sound,Neuromancer,NaN,-1.0,Om Yoga,...,1,3,1,False,True,False,True,False,True,0.0


In [23]:
song_length_mode = test_all_data.mode().iloc[0]['song_length']
test_all_data['song_length'] = test_all_data['song_length'].fillna(song_length_mode)

In [24]:
test_dl = learn.dls.test_dl(test_all_data)
output = learn.get_preds(dl=test_dl, with_decoded=True)
output

(tensor([[0.3491],
         [0.3317],
         [0.1528],
         ...,
         [0.4176],
         [0.1978],
         [0.2489]]),
 None,
 tensor([[0.3491],
         [0.3317],
         [0.1528],
         ...,
         [0.4176],
         [0.1978],
         [0.2489]]))

In [25]:
probs,_,indexes = output

In [26]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

,id,target
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
2556785,2556785,0.5
2556786,2556786,0.5
2556787,2556787,0.5
2556788,2556788,0.5


In [27]:
probs = torch.clip(probs, 0, 1)
test_all_data['target'] = probs

sub_df = test_all_data[['id','target']]
sub_df.to_csv(f'submission.csv', index=False)

In [28]:
!head submission.csv

id,target
0,0.34909016
1,0.33172256
2,0.15281528
3,0.17462546
4,0.15281528
5,0.29598725
6,0.15281528
7,0.55729073
8,0.5432894


In [29]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'kkbox-recommendation-6', comp)

100%|██████████| 44.2M/44.2M [00:00<00:00, 51.3MB/s]


In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
